# Training data creation

In [34]:
import numpy as np
import pandas as pd
from numpy.typing import NDArray

from keras.models import Sequential, Model
from keras.layers import Dense
from keras.optimizers import Adam
import keras.losses as losses
import keras.activations as activations
import keras.metrics as metrics

import itertools as it
from collections.abc import Iterable
from typing import Callable, TypeVar, cast

Create every combination of moves for our input training data

In [2]:
moves = ["rock", "paper", "scissors"]
combs = pd.DataFrame(it.product(moves, repeat=2), columns=["p1", "p2"])
display(combs)

,p1,p2
0,rock,rock
1,rock,paper
2,rock,scissors
3,paper,rock
4,paper,paper
5,paper,scissors
6,scissors,rock
7,scissors,paper
8,scissors,scissors


hot-encode inputs

In [3]:
encoded_p1 = pd.get_dummies(combs['p1']).values
encoded_p2 = pd.get_dummies(combs['p2']).values
display(encoded_p1)
display(encoded_p2)

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]], dtype=uint8)

array([[0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1]], dtype=uint8)

calculate scores for each of our possible input; we are gonna use it for our Y data

In [4]:
to_number = lambda arr: arr@range(len(moves))
p1_moves = np.array(list(map(to_number, encoded_p1)))
p2_moves = np.array(list(map(to_number, encoded_p2)))
display(p1_moves)
display(p2_moves)
output = (p1_moves-p2_moves)%3
display(output)

I = np.identity(len(moves))
to_array = lambda x: I[x]
output = np.array(list(map(to_array, output)))
display(output)

array([1, 1, 1, 0, 0, 0, 2, 2, 2])

array([1, 0, 2, 1, 0, 2, 1, 0, 2])

array([0, 1, 2, 2, 0, 1, 1, 2, 0])

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

Finally, arrange data together

In [6]:
x = pd.get_dummies(combs).values
display(pd.get_dummies(combs))
display(x)
y = output
display(y)
display(len(x)==len(y))

,p1_paper,p1_rock,p1_scissors,p2_paper,p2_rock,p2_scissors
0,0,1,0,0,1,0
1,0,1,0,1,0,0
2,0,1,0,0,0,1
3,1,0,0,0,1,0
4,1,0,0,1,0,0
5,1,0,0,0,0,1
6,0,0,1,0,1,0
7,0,0,1,1,0,0
8,0,0,1,0,0,1


array([[0, 1, 0, 0, 1, 0],
       [0, 1, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 1],
       [1, 0, 0, 0, 1, 0],
       [1, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 1, 0],
       [0, 0, 1, 1, 0, 0],
       [0, 0, 1, 0, 0, 1]], dtype=uint8)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

True

In [13]:
T = TypeVar('T')
U = TypeVar('U', bound=np.generic, covariant=True)


def map_array(fn: Callable[[T], U], arr: Iterable[T]) -> NDArray[U]:
    """maps an iterable to a numpy array using provided function"""
    return np.array(list(map(fn, arr)))


def training_data() -> tuple[NDArray, NDArray]:
    # written using jupyter notebook

    moves = ["rock", "paper", "scissors"]
    # create all possible combinations
    combs = pd.DataFrame(it.product(moves, repeat=2), columns=["p1", "p2"])

    # convert string values into hot-encoded arrays
    encoded_p1 = pd.get_dummies(combs['p1']).values
    encoded_p2 = pd.get_dummies(combs['p2']).values

    # convert the arrays into scalar arrays,
    # calculate output using matrix substraction and scalar modulo
    to_number = lambda arr: arr@range(len(moves))
    p1_moves = map_array(to_number, encoded_p1)
    p2_moves = map_array(to_number, encoded_p2)
    output = (p1_moves-p2_moves)%3  # type: ignore

    # convert output to hot-encoded arrays
    I = np.identity(len(moves))
    to_array = lambda x: I[x]
    output = map_array(to_array, output)

    return pd.get_dummies(combs).values, output

x, y = training_data()

display(x, y)

array([[0, 1, 0, 0, 1, 0],
       [0, 1, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 1],
       [1, 0, 0, 0, 1, 0],
       [1, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 1, 0],
       [0, 0, 1, 1, 0, 0],
       [0, 0, 1, 0, 0, 1]], dtype=uint8)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [76]:
def compile_model():
    model = Sequential()
    model.add(Dense(6, input_dim=6, activation=activations.relu))
    model.add(Dense(16, input_dim=6, activation=activations.relu))
    model.add(Dense(3, activation=activations.softmax))
    adam = Adam(learning_rate=0.01)
    model.compile(loss=losses.CategoricalCrossentropy(), optimizer=adam, metrics=[metrics.categorical_accuracy])
    return model

model = compile_model()
display(model)

In [77]:
def train(model: Model, x: NDArray, y: NDArray, epochs: int = 10) -> None:
    model.fit(x, y, epochs=epochs, verbose=1)  # type: ignore

train(model, x, y, epochs=200)

Epoch 1/200
1/1 [==============================] - 0s 153ms/step - loss: 1.1123 - categorical_accuracy: 0.3333
Epoch 2/200
1/1 [==============================] - 0s 3ms/step - loss: 1.0939 - categorical_accuracy: 0.3333
Epoch 3/200
1/1 [==============================] - 0s 2ms/step - loss: 1.0809 - categorical_accuracy: 0.5556
Epoch 4/200
1/1 [==============================] - 0s 2ms/step - loss: 1.0703 - categorical_accuracy: 0.5556
Epoch 5/200
1/1 [==============================] - 0s 2ms/step - loss: 1.0616 - categorical_accuracy: 0.5556
Epoch 6/200
1/1 [==============================] - 0s 29ms/step - loss: 1.0534 - categorical_accuracy: 0.5556
Epoch 7/200
1/1 [==============================] - 0s 8ms/step - loss: 1.0452 - categorical_accuracy: 0.6667
Epoch 8/200
1/1 [==============================] - 0s 3ms/step - loss: 1.0372 - categorical_accuracy: 0.6667
Epoch 9/200
1/1 [==============================] - 0s 11ms/step - loss: 1.0295 - categorical_accuracy: 0.5556
Epoch 10/200
1/

In [105]:

def test(model: Model, x, y, verbose=False) -> tuple[NDArray, bool]:
    prediction: NDArray = model.predict(x)
    prediction_rounded = prediction.round(decimals=1)
    if verbose:
        prediction_formatted = map_array(lambda x: [f"{i:.2f}" for i in x], prediction)
        print(prediction_formatted)
    return prediction_rounded, (prediction_rounded==y).all()

print(test(model, x, y))

1/1 [==============================] - 0s 24ms/step
(array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32), True)
